Purpose: Run hierarchical clustering of treatments from log2 TPM data corrected for BioProject using pyComBat (both the full dataset & photosynthetic tissues only).<br>
Author: Anna Pardo<br>
Date initiated: Feb. 29, 2024

In [1]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
import statistics

In [ ]:
# load corrected TPM matrices
bpcor_all_tpm = 